In [1]:
# import os, pyodbc, time, unidecode
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [250]:
sql =\
"""
select BranchID, sq.DiscID, sq.DiscSeq, sq.Descr, sq.DiscIDPN, OrderNbr, LineRef, SOLineRef, 
case when GroupRefLineRef = '' then SOLineRef else GroupRefLineRef end as GroupRefLineRef,
dis.Crtd_DateTime,
dis.LUpd_DateTime
from OM_OrdDisc dis
INNER JOIN dbo.OM_DiscSeq sq WITH (NOLOCK)
ON sq.DiscID = dis.DiscID
AND sq.DiscSeq = dis.DiscSeq
where CAST(dis.Crtd_DateTime as date)>= '2021-05-01'  and cast (dis.Crtd_DateTime as date)<= '2021-05-31'
"""

In [253]:
df = get_ms_df(sql)

In [254]:
# df.to_clipboard()

In [255]:
df = pd.concat([df.GroupRefLineRef.str.split(",", expand=True), df], axis=1)

In [256]:
# df.shape

In [257]:
# df1 = df.head(200).copy()

In [258]:
df_aut = pd.melt(df,id_vars=['BranchID', 'DiscID', 'DiscSeq','Descr', 'DiscIDPN', 'OrderNbr', 'LineRef', 'SOLineRef', 'GroupRefLineRef', 'Crtd_DateTime', 'LUpd_DateTime'], value_name='GLineRef')

In [259]:
df_aut.shape

(215514, 13)

In [260]:
df=df_aut[df_aut['GLineRef'].notna()].copy()

In [262]:
# df.isna().sum()

In [263]:
# df_aut.to_clipboard()

In [264]:
df['key1'] = df.BranchID.astype(str)+"-"+df.OrderNbr.astype(str)+"-"+df.GLineRef.astype(str)
df['key2'] = df.BranchID.astype(str)+"-"+df.OrderNbr.astype(str)+"-"+df.GLineRef.astype(str)+"-"+df.DiscIDPN.astype(str)

In [211]:
# drop_cols(df_aut, ['variable'])

In [212]:
# df['key'].isna().sum()

In [265]:
df = dropdup(df,1, ['key2'])

In [215]:
# df_up=df[checkdup(df, 2, subset=['key1'])]
# df_aut.to_clipboard()

In [272]:
df1 = df[['key1', 'DiscIDPN']]
df2 = df[['key1', 'Descr']]

In [273]:
df3=df1.groupby(['key1'])['DiscIDPN'].apply(list).reset_index(name='DiscIDPN')

In [275]:
df4=df2.groupby(['key1'])['Descr'].apply(list).reset_index(name='Descr')

In [277]:
df4

,key1,Descr
0,MR0001-CO052021-00006-00001,[MR- CT hỗ trợ bán hàng CLC3]
1,MR0001-CO052021-00012-00001,[MR- CT hỗ trợ bán hàng PKK]
2,MR0001-CO052021-00014-00001,[MR- CT Giỏ hàng chiến lược OTC Quý 2]
3,MR0001-CO052021-00016-00001,[MR- CT Giỏ hàng chiến lược OTC Quý 2]
4,MR0001-CO052021-00016-00002,[MR- CT Giỏ hàng chiến lược OTC Quý 2]
...,...,...
30850,MR0016-RP052021-00001-00003,[MR- CT Giỏ hàng chiến lược OTC Quý 2]
30851,MR0016-RP052021-00001-00004,[MR- CT Giỏ hàng chiến lược OTC Quý 2]
30852,MR0016-RP052021-00001-00005,[MR- CT Giỏ hàng chiến lược OTC Quý 2]
30853,MR0016-RP052021-00001-00006,[MR- CT Giỏ hàng chiến lược OTC Quý 2]


In [268]:
df2.head()

,DiscIDPN
key1,
MR0001-CO052021-00006-00001,"[C, S, B, H, 2, 1, 0, 2, C, L, C, 3, -, 0, 5, ..."
MR0001-CO052021-00012-00001,"[C, S, B, H, 2, 1, 0, 2, P, K, K, -, 0, 6, Q, ..."
MR0001-CO052021-00014-00001,"[C, T, K, M, 2, 1, 0, 4, O, T, C, -, S, O, 1, ..."
MR0001-CO052021-00016-00001,"[C, T, K, M, 2, 1, 0, 4, O, T, C, -, S, O, 1, ..."
MR0001-CO052021-00016-00002,"[C, T, K, M, 2, 1, 0, 4, O, T, C, -, S, O, 1, ..."


In [242]:
df2['DiscIDPN1'] = df2['DiscIDPN'].apply(lambda x:x[0])

In [247]:
df2['DiscIDPN2'] = df2['DiscIDPN'].apply(lambda x:x[1] if(len(x) > 1) else None)
df2['DiscIDPN3'] = df2['DiscIDPN'].apply(lambda x:x[2] if(len(x) > 2) else None)

In [248]:
df2.to_clipboard()

In [217]:
vc(df_up, 'key1').to_clipboard()

In [14]:
df.to_excel("excel.xlsx")

In [2]:
# day_ago = 2
datenow = datetime.now().strftime("%Y%m%d")
# datenow_day_ago = ( datetime.now()-timedelta(day_ago) ).strftime("%Y%m%d")
# param_1 = f"'{datenow_day_ago}'"
param_2 = f"'20220301'"
param_3 = f"'{datenow}'"
param_4 = f"'20220331'"


In [3]:
thang = 'T3'

In [2]:
query = "EXEC [pr_OM_RawdataSellOutPayroll_BI_v1] @Fromdate='20220416', @Todate='20220416'"
FINAL = get_ms_df(sql=query)

In [3]:
FINAL.shape

(1454, 54)

In [5]:
FINAL.columns = cleancols(FINAL)

In [6]:
# FINAL.SoLuong.sum()

In [7]:
if FINAL.shape[0] != 0:
    print('do it')
else:
    print('not do it')

do it


In [8]:
# dk = FINAL.SoDonDatHang == 'DH3-0222-02307'

In [9]:
# FINAL[dk].to_clipboard()

In [10]:
# lst = ['MaCongTyCN','SoDonDatHang','MaSanPham','SoLo','SoLuong','DonGiaChuaVAT','DoanhSoChuaVAT']

In [11]:
# FINAL[lst].to_clipboard()

In [12]:
query = f"EXEC [pr_OM_RawdataSellOutPayroll_BI_FreeItem] @Fromdate={param_2}, @Todate={param_4}"
ITEM = get_ms_df(sql=query)

In [13]:
# FINAL.columns

In [14]:
left_on=['MaCongTyCN','LineRef','MaHD']
right_on = ['BranchID','SOLineRef','OrderNbr']
# ITEM.columns

In [15]:
FINAL2 = FINAL.merge(ITEM, left_on=left_on, right_on=right_on, how='left')

In [16]:
# FINAL2.to_csv("FINAL2.csv", index=False)

In [17]:
query = f"EXEC [pr_OM_RawdataSellOutPayroll_BI_FreeAmount] @Fromdate={param_2}, @Todate={param_4}"
AMOUNT = get_ms_df(sql=query)

In [18]:
AMOUNT.columns = ['amt_'+str(e) for e in AMOUNT.columns]

In [19]:
# AMOUNT.columns

In [20]:
left_on=['MaCongTyCN','LineRef','MaHD']
right_on = ['amt_BranchID','amt_LineRef','amt_OrderNbr']
how='left'

In [21]:
FINAL3 = FINAL2.merge(AMOUNT, left_on=left_on, right_on=right_on, how=how)

In [22]:
# FINAL3.to_csv("FINAL3.csv", index=False)

In [23]:
lst = [
'MaCongTyCN', 'CongTyCN', 'NgayChungTu', 'SoDonDatHang',
'MaKHDMS', 'TenKhachHang', 'TenTinhKH',
'TenQuanHUyen', 'MaKenhKH', 'MaKenhPhu', 'MaSanPham', 'TenSanPhamNB',
'TenSanPhamVietTat', 'SoLuong', 'DonGiaCoVAT',
'DoanhSoCoVAT', 'DonGiaChuaVAT', 'DoanhSoChuaVAT', 
'NgayGiaoHang', 'MaNV', 'TenCVBH', 'TenQuanLyTT', 'TenQuanLyKhuVuc',
'TenQuanLyVung', 'MaNVGH', 'NguoiGiaohang', 'TrangThaiGiaoHang',
'DonViGiaoHang', 'TenNhaVanChuyen', 'KieuDonHang', 'FreeItemID', 'FreeItemQty', 'TypeDiscount',
'DiscAmt', 'DiscPct', 'DiscIDPN', 'DiscID', 'DiscSeq',
'Descr', 'amt_InvtID',
'amt_FreeItemQty', 'amt_TypeDiscount', 'amt_DiscAmt',
'amt_DiscPct', 'amt_DiscIDPN', 'amt_DiscID', 'amt_DiscSeq',
'amt_Descr'
]

In [24]:
FINAL3 = FINAL3[lst].copy()

In [25]:
FINAL3.FreeItemID.fillna('',inplace=True)
FINAL3.amt_InvtID.fillna('',inplace=True)

In [26]:
FINAL3['FreeItemID']=FINAL3.FreeItemID + FINAL3.amt_InvtID
FINAL3.FreeItemQty.fillna(0,inplace=True)
FINAL3.amt_FreeItemQty.fillna(0,inplace=True)
FINAL3['FreeItemQty']=FINAL3.FreeItemQty + FINAL3.amt_FreeItemQty

In [27]:
FINAL3.TypeDiscount.fillna('',inplace=True)
FINAL3.amt_TypeDiscount.fillna('',inplace=True)
FINAL3['TypeDiscount']=FINAL3.TypeDiscount + FINAL3.amt_TypeDiscount

In [28]:
FINAL3.DiscID.fillna('',inplace=True)
FINAL3.amt_DiscID.fillna('',inplace=True)
FINAL3['DiscID']=FINAL3.DiscID + FINAL3.amt_DiscID

In [29]:
FINAL3.DiscSeq.fillna('',inplace=True)
FINAL3.amt_DiscSeq.fillna('',inplace=True)
FINAL3['DiscSeq']=FINAL3.DiscSeq + FINAL3.amt_DiscSeq

In [30]:
FINAL3.Descr.fillna('',inplace=True)
FINAL3.amt_Descr.fillna('',inplace=True)
FINAL3['Descr']=FINAL3.Descr + FINAL3.amt_Descr

In [31]:
FINAL3.DiscAmt.fillna(0,inplace=True)
FINAL3.amt_DiscAmt.fillna(0,inplace=True)
FINAL3['DiscAmt']=FINAL3.DiscAmt + FINAL3.amt_DiscAmt

In [32]:
FINAL3.DiscPct.fillna(0,inplace=True)
FINAL3.amt_DiscPct.fillna(0,inplace=True)
FINAL3['DiscPct']=FINAL3.DiscPct + FINAL3.amt_DiscPct

In [33]:
FINAL3.DiscIDPN.fillna('',inplace=True)
FINAL3.amt_DiscIDPN.fillna('',inplace=True)
FINAL3['DiscIDPN']=FINAL3.DiscIDPN + FINAL3.amt_DiscIDPN

In [34]:
lst = \
    ['MaCongTyCN', 'CongTyCN', 'NgayChungTu', 'SoDonDatHang', 'MaKHDMS',
       'TenKhachHang', 'TenTinhKH', 'TenQuanHUyen', 'MaKenhKH', 'MaKenhPhu',
       'MaSanPham', 'TenSanPhamNB', 'TenSanPhamVietTat', 'SoLuong',
       'DonGiaCoVAT', 'DoanhSoCoVAT', 'DonGiaChuaVAT', 'DoanhSoChuaVAT',
       'NgayGiaoHang', 'MaNV', 'TenCVBH', 'TenQuanLyTT', 'TenQuanLyKhuVuc',
       'TenQuanLyVung', 'MaNVGH', 'NguoiGiaohang', 'TrangThaiGiaoHang',
       'DonViGiaoHang', 'TenNhaVanChuyen', 'KieuDonHang', 'FreeItemID',
       'FreeItemQty', 'TypeDiscount', 'DiscAmt', 'DiscPct', 'DiscIDPN',
       'DiscID', 'DiscSeq', 'Descr']

In [35]:
FINAL3 = FINAL3[lst].copy()

In [36]:
FINAL3['DiscFreeItemAmt'] = np.where(FINAL3['FreeItemQty'] == 0, 0, FINAL3['SoLuong'] * FINAL3['DonGiaChuaVAT'])

In [37]:
# FINAL3.to_csv(f"FINAL3_{thang}_Detail.csv", index=False)

In [38]:
# FINAL3 = pivot(FINAL3, ['MaKHDMS','TenKhachHang','TenTinhKH','MaKenhKH','MaKenhPhu','MaSanPham','TenSanPhamNB','MaNV','TenCVBH'], {'SoLuong':np.sum,'DoanhSoChuaVAT':np.sum,'DiscAmt':np.sum, 'DiscFreeItemAmt':np.sum})

In [39]:
FINAL3['Thang'] = thang

In [120]:
# FINAL3.ngaychungtu

In [40]:
execute_values_insert(FINAL3, "f_temp_sales_with_promo")

Data inserted using execute_values() successfully..


In [4]:
sql = \
'''
select BranchID, CustID, SUM(Amt) as Total_Amt, CAST(a.Crtd_DateTime as DATE) as CreateDate from OM_SalesOrdAccumulate AS a
INNER JOIN OM_Accumulated AS b ON
a.AccumulateID = b.AccumulateID
and b.FromDate >= '2022-01-01'
WHERE a.Crtd_DateTime > '2022-01-01' and a.Crtd_DateTime <= '2022-03-31'
GROUP BY BranchID, CustID, CAST (a.Crtd_DateTime as DATE)
'''

In [5]:
AC = get_ms_df(sql=sql)

In [7]:
execute_values_insert(AC, "f_temp_ac_promo")

Data inserted using execute_values() successfully..


In [357]:
FINAL3.to_csv(f"FINAL3_{thang}.csv", index=False)

In [122]:
# query = f"EXEC [pr_OM_RawdataSellOutPayroll_BI_v1] @Fromdate={param_1}, @Todate={param_2}"
# get_ms_csv(query, 'sales_2.csv')
# FINAL1 = pd.read_csv('sales_2.csv')
# FINAL = pd.read_csv('sales.csv')
FINAL.columns = cleancols(FINAL)
# FINAL.columns
# FINAL.DoanhSoChuaVAT.sum()
if str(FINAL.NgayGiaoHang.dtypes) == 'object' : FINAL.NgayGiaoHang = pd.to_datetime(FINAL.NgayGiaoHang, dayfirst=True)
if str(FINAL.NgayTraHang.dtypes) == 'object' : FINAL.NgayTraHang = pd.to_datetime(FINAL.NgayTraHang, dayfirst=True)
FINAL.NgayGiaoHang.fillna(datetime(1900, 1, 1), inplace=True)

FINAL['phanloaispcl'] = FINAL['MaSanPham'].map(
    df_to_dict(get_ps_df("select masanpham, phanloai from d_nhom_sp where nhomsp='SPCL'"))
).fillna('Khác')

FINAL['nhomsp'] = FINAL['MaSanPham'].map(
    df_to_dict(get_ps_df("select masanpham, nhomsp from d_nhom_sp where nhomsp IN ('SPCL', 'SP MOI') "))
).fillna('Khác')

FINAL['khuvucviettat'] = FINAL['TenKhuVuc'].map(
    df_to_dict(get_ps_df("select * from d_mkv_viet_tat"))
)

FINAL['chinhanh'] = FINAL['MaCongTyCN'].map(
    df_to_dict(get_ps_df("select * from d_chi_nhanh"))
)

FINAL['newhco'] = (FINAL['MaKenhPhu']+FINAL['MaPhanLoaiHCO']).map(
    df_to_dict(get_ps_df("SELECT concat(makenhphu, maphanloaihco) as concat, new_mahco FROM d_pl_hco"))
)

FINAL['phanam'] = FINAL['MaSanPham'].map(
    df_to_dict(get_ps_df("select masanpham, nhomsp from d_nhom_sp where nhomsp='PHA NAM'"))).fillna('Merap')

FINAL['thang'] = FINAL['NgayChungTu'] + pd.offsets.Day() - pd.offsets.MonthBegin()

FINAL['inserted_at'] = datetime.now()

In [12]:
FINAL.columns = cleancols(FINAL)
FINAL.columns = lower_col(FINAL)

In [13]:
pk = ['macongtycn', 'ngaychungtu', 'sodondathang', 'masanpham', 'solo', 'lineref', 'soluong']

In [14]:
FINAL[checkdup(FINAL,2,pk)].to_clipboard()

In [9]:
# FINAL['inserted_at'] = datetime(2022,1,26)

pk = ['macongtycn', 'ngaychungtu', 'sodondathang', 'masanpham', 'solo', 'lineref', 'soluong']

execute_values_upsert(FINAL, 'f_sales', pk=pk)

INSERT INTO f_sales(MaCongTyCN,CongTyCN,NgayChungTu,SoDonDatHang,MaHD,SoDonTraHang,NgayTraHang,HoaDon,TrangThai,MaKHThue,MaKHCu,TenKhachHang,TenVungBH,TenKhuVuc,TenTinhKH,TenQuanHUyen,MaKenhKH,TenKenhKH,MaKenhPhu,TenKenhPhu,MaHCO,TenHCO,MaPhanLoaiHCO,TenPhanLoaiHCO,MaPhanHangHCO,TenPhanHangHCO,NhanHang,MaSanPham,TenSanPhamNB,TenSanPhamVietTat,SoLo,LineRef,SoLuong,DonGiaCoVAT,DoanhSoCoVAT,DonGiaChuaVAT,DoanhSoChuaVAT,NgayDatDon,NgayGiaoHang,MaNV,TenCVBH,TenQuanLyTT,TenQuanLyKhuVuc,TenQuanLyVung,MaNVGH,NguoiGiaohang,TrangThaiGiaoHang,DonViGiaoHang,TenNhaVanChuyen,KieuDonHang,MaKho,TenKho,phanloaispcl,nhomsp,khuvucviettat,chinhanh,newhco,phanam,thang,inserted_at) VALUES %s ON CONFLICT (macongtycn, ngaychungtu, sodondathang, masanpham, solo, lineref, soluong) DO UPDATE SET MaCongTyCN = excluded.MaCongTyCN, CongTyCN = excluded.CongTyCN, NgayChungTu = excluded.NgayChungTu, SoDonDatHang = excluded.SoDonDatHang, MaHD = excluded.MaHD, SoDonTraHang = excluded.SoDonTraHang, NgayTraHang = excluded

In [4]:
FINAL.columns = cleancols(FINAL)

FINAL.NgayGiaoHang.fillna(datetime(1900, 1, 1), inplace=True)

FINAL['phanloaispcl'] = FINAL['MaSanPham'].map(
    df_to_dict(get_ps_df("select masanpham, phanloai from d_nhom_sp where nhomsp='SPCL'"))
).fillna('Khác')

FINAL['nhomsp'] = FINAL['MaSanPham'].map(
    df_to_dict(get_ps_df("select masanpham, nhomsp from d_nhom_sp where nhomsp IN ('SPCL', 'SP MOI') "))
).fillna('Khác')

FINAL['khuvucviettat'] = FINAL['TenKhuVuc'].map(
    df_to_dict(get_ps_df("select * from d_mkv_viet_tat"))
)

FINAL['chinhanh'] = FINAL['MaCongTyCN'].map(
    df_to_dict(get_ps_df("select * from d_chi_nhanh"))
)

FINAL['newhco'] = (FINAL['MaKenhPhu']+FINAL['MaPhanLoaiHCO']).map(
    df_to_dict(get_ps_df("SELECT concat(makenhphu, maphanloaihco) as concat, new_mahco FROM d_pl_hco"))
)

FINAL['phanam'] = FINAL['MaSanPham'].map(
    df_to_dict(get_ps_df("select masanpham, nhomsp from d_nhom_sp where nhomsp='PHA NAM'"))).fillna('Merap')

FINAL['thang'] = FINAL['NgayChungTu'] + pd.offsets.Day() - pd.offsets.MonthBegin()

FINAL['inserted_at'] = datetime(2022,2,7)

pk = ['macongtycn', 'ngaychungtu', 'sodondathang', 'masanpham', 'solo', 'lineref', 'soluong']

execute_values_upsert(FINAL, 'f_sales', pk=pk)

INSERT INTO f_sales(MaCongTyCN,CongTyCN,NgayChungTu,SoDonDatHang,SoDonTraHang,NgayTraHang,HoaDon,TrangThai,MaKHThue,MaKHCu,TenKhachHang,TenVungBH,TenKhuVuc,TenTinhKH,TenQuanHUyen,MaKenhKH,TenKenhKH,MaKenhPhu,TenKenhPhu,MaHCO,TenHCO,MaPhanLoaiHCO,TenPhanLoaiHCO,MaPhanHangHCO,TenPhanHangHCO,NhanHang,MaSanPham,TenSanPhamNB,TenSanPhamVietTat,SoLo,LineRef,SoLuong,DonGiaCoVAT,DoanhSoCoVAT,DonGiaChuaVAT,DoanhSoChuaVAT,NgayDatDon,NgayGiaoHang,MaNV,TenCVBH,TenQuanLyTT,TenQuanLyKhuVuc,TenQuanLyVung,MaNVGH,NguoiGiaohang,TrangThaiGiaoHang,DonViGiaoHang,TenNhaVanChuyen,KieuDonHang,MaKho,TenKho,phanloaispcl,nhomsp,khuvucviettat,chinhanh,newhco,phanam,thang,inserted_at) VALUES %s ON CONFLICT (macongtycn, ngaychungtu, sodondathang, masanpham, solo, lineref, soluong) DO UPDATE SET MaCongTyCN = excluded.MaCongTyCN, CongTyCN = excluded.CongTyCN, NgayChungTu = excluded.NgayChungTu, SoDonDatHang = excluded.SoDonDatHang, SoDonTraHang = excluded.SoDonTraHang, NgayTraHang = excluded.NgayTraHang, HoaDon = excl

In [6]:
# FINAL.NgayGiaoHang.fillna(datetime(1900, 1, 1), inplace=True)

In [7]:
# vc(FINAL, 'NgayGiaoHang').to_clipboard()

In [8]:
# FINAL.dtypes

In [9]:
# FINAL.columns = lower_col(FINAL)

In [11]:
FINAL.columns

Index(['MaCongTyCN', 'CongTyCN', 'NgayChungTu', 'SoDonDatHang', 'SoDonTraHang',
       'NgayTraHang', 'HoaDon', 'TrangThai', 'MaKHThue', 'MaKHCu',
       'TenKhachHang', 'TenVungBH', 'TenKhuVuc', 'TenTinhKH', 'TenQuanHUyen',
       'MaKenhKH', 'TenKenhKH', 'MaKenhPhu', 'TenKenhPhu', 'MaHCO', 'TenHCO',
       'MaPhanLoaiHCO', 'TenPhanLoaiHCO', 'MaPhanHangHCO', 'TenPhanHangHCO',
       'NhanHang', 'MaSanPham', 'TenSanPhamNB', 'TenSanPhamVietTat', 'SoLo',
       'LineRef', 'SoLuong', 'DonGiaCoVAT', 'DoanhSoCoVAT', 'DonGiaChuaVAT',
       'DoanhSoChuaVAT', 'NgayDatDon', 'NgayGiaoHang', 'MaNV', 'TenCVBH',
       'TenQuanLyTT', 'TenQuanLyKhuVuc', 'TenQuanLyVung', 'MaNVGH',
       'NguoiGiaohang', 'TrangThaiGiaoHang', 'DonViGiaoHang',
       'TenNhaVanChuyen', 'KieuDonHang', 'MaKho', 'TenKho', 'phanloaispcl',
       'nhomsp', 'khuvucviettat', 'chinhanh', 'newhco', 'phanam', 'thang',
       'inserted_at'],
      dtype='object')

In [10]:
FINAL.columns

Index(['MaCongTyCN', 'CongTyCN', 'NgayChungTu', 'SoDonDatHang', 'SoDonTraHang',
       'NgayTraHang', 'HoaDon', 'TrangThai', 'MaKHThue', 'MaKHCu',
       'TenKhachHang', 'TenVungBH', 'TenKhuVuc', 'TenTinhKH', 'MaKenhKH',
       'TenKenhKH', 'MaKenhPhu', 'TenKenhPhu', 'MaHCO', 'TenHCO',
       'MaPhanLoaiHCO', 'TenPhanLoaiHCO', 'MaPhanHangHCO', 'TenPhanHangHCO',
       'NhanHang', 'MaSanPham', 'TenSanPhamNB', 'TenSanPhamVietTat', 'SoLo',
       'LineRef', 'SoLuong', 'DonGiaCoVAT', 'DoanhSoCoVAT', 'DonGiaChuaVAT',
       'DoanhSoChuaVAT', 'NgayDatDon', 'NgayGiaoHang', 'MaNV', 'TenCVBH',
       'TenQuanLyTT', 'TenQuanLyKhuVuc', 'TenQuanLyVung', 'MaNVGH',
       'NguoiGiaohang', 'TrangThaiGiaoHang', 'DonViGiaoHang',
       'TenNhaVanChuyen', 'KieuDonHang', 'MaKho', 'TenKho', 'phanloaispcl',
       'nhomsp', 'khuvucviettat', 'chinhanh', 'newhco', 'phanam', 'thang',
       'inserted_at'],
      dtype='object')

In [11]:
pk = ['macongtycn', 'ngaychungtu', 'sodondathang', 'masanpham', 'solo', 'lineref', 'soluong']

In [12]:
# df_filter(FINAL, sodondathang='OO102021-00001').to_clipboard()

In [13]:
execute_values_upsert(FINAL, 'f_sales', pk=pk)

INSERT INTO f_sales(MaCongTyCN,CongTyCN,NgayChungTu,SoDonDatHang,SoDonTraHang,NgayTraHang,HoaDon,TrangThai,MaKHThue,MaKHCu,TenKhachHang,TenVungBH,TenKhuVuc,TenTinhKH,MaKenhKH,TenKenhKH,MaKenhPhu,TenKenhPhu,MaHCO,TenHCO,MaPhanLoaiHCO,TenPhanLoaiHCO,MaPhanHangHCO,TenPhanHangHCO,NhanHang,MaSanPham,TenSanPhamNB,TenSanPhamVietTat,SoLo,LineRef,SoLuong,DonGiaCoVAT,DoanhSoCoVAT,DonGiaChuaVAT,DoanhSoChuaVAT,NgayDatDon,NgayGiaoHang,MaNV,TenCVBH,TenQuanLyTT,TenQuanLyKhuVuc,TenQuanLyVung,MaNVGH,NguoiGiaohang,TrangThaiGiaoHang,DonViGiaoHang,TenNhaVanChuyen,KieuDonHang,MaKho,TenKho,phanloaispcl,nhomsp,khuvucviettat,chinhanh,newhco,phanam,thang,inserted_at) VALUES %s ON CONFLICT (macongtycn, ngaychungtu, sodondathang, masanpham, solo, lineref, soluong) DO UPDATE SET MaCongTyCN = excluded.MaCongTyCN, CongTyCN = excluded.CongTyCN, NgayChungTu = excluded.NgayChungTu, SoDonDatHang = excluded.SoDonDatHang, SoDonTraHang = excluded.SoDonTraHang, NgayTraHang = excluded.NgayTraHang, HoaDon = excluded.HoaDon, 

In [39]:
# FINAL2 = FINAL.head()

In [40]:
# FINAL2.to_csv('FINAL.csv')

In [24]:
############# ENDDDDDDDDDDD #################

In [2]:
df_nhansu = pd.read_csv('datanhansu.csv')

In [3]:
df_nhansu.columns = cleancols(df_nhansu)

In [4]:
print_df_schema(df_nhansu)

STT BIGINT NULL,
PHONG VARCHAR(100) NULL,
BP VARCHAR(100) NULL,
MaNVCRSCRSS VARCHAR(100) NULL,
HOTEN VARCHAR(100) NULL,
HOTENKHONGDAU VARCHAR(100) NULL,
CHUCDANH VARCHAR(100) NULL,
TINHTRANG VARCHAR(100) NULL,
Ngayvaocongty VARCHAR(100) NULL,
KV VARCHAR(100) NULL,
SDT VARCHAR(100) NULL,
EMAIL VARCHAR(100) NULL,
MANVQLTT VARCHAR(100) NULL,
QUANLYTRUCTIEP VARCHAR(100) NULL,
MANVQLV VARCHAR(100) NULL,
QLKHUVUC VARCHAR(100) NULL,
MANVQLV1 VARCHAR(100) NULL,
QLVUNG VARCHAR(100) NULL,


In [10]:
df_nhansu.columns

Index(['STT', 'PHONG', 'BP', 'MaNVCRSCRSS', 'HOTEN', 'HOTENKHONGDAU',
       'CHUCDANH', 'TINHTRANG', 'Ngayvaocongty', 'KV', 'SDT', 'EMAIL',
       'MANVQLTT', 'QUANLYTRUCTIEP', 'MANVQLV', 'QLKHUVUC', 'MANVQLV.1',
       'QLVUNG'],
      dtype='object')

In [5]:
insert_df_to_postgres(
postgres_conf, tbl_name="nhansu",
df=df_nhansu, primary_keys=['manvcrscrss'], schema="biteam")

In [2]:
# df_dict = pd.read_csv('ten_tat_mkv.csv', index_col='TenKhuVuc')
df_mkv_viet_tat = pd.read_csv('ten_tat_mkv.csv')

In [3]:
print_df_schema(df_mkv_viet_tat)

TenKhuVuc VARCHAR(100) NULL,
KhuVuc VARCHAR(100) NULL,


In [7]:
x = [x.lower() for x in df_mkv_viet_tat.columns]
df_mkv_viet_tat.columns = x
insert_df_to_postgres(df=df_mkv_viet_tat, postgres_conf=postgres_conf, tbl_name='mkv_viet_tat',schema="biteam", primary_keys=['tenkhuvuc'])

In [1]:
t1 = time.perf_counter()
now = datetime.now()
print('start program %s' % now)
print('current location %s' % os.getcwd())

# Server config
server = '115.165.164.234'
driver = 'SQL Server'
db1 = 'PhaNam_eSales_PRO'
tcon = 'no'
uname = 'duyvq'
pword = '123VanQuangDuy'
cnxn = pyodbc.connect(driver='{ODBC Driver 17 for SQL Server}',
                      host=server, database=db1, trusted_connection=tcon,
                      user=uname, password=pword)
cursor = cnxn.cursor()

# Load df from server
print('Load df from server', datetime.now())
datenow = datetime.now().strftime("%Y%m%d")
param_1 = f"'20210901'"
param_2 = f"'20210930'"
query = f"EXEC [pr_OM_RawdataSellOutPayroll_BI] @Fromdate={param_1}, @Todate={param_2}"
PATH = "D:\\data_sale\\ipython_file\\data_raw_thang\\"
DATAPATH = PATH + "data\\"
FILE = f"data\\raw_data_old_{param_2}.csv"

# df1 = pd.read_sql(query, cnxn)
cursor.close()
cnxn.close()
# df1.to_csv(PATH + FILE)

start program 2021-10-08 12:32:26.251163
current location d:\data_sale\ipython_file\data_raw_thang
Load df from server 2021-10-08 12:32:26.362819


In [4]:
PATH = "D:\\data_sale\\ipython_file\\data_raw_thang\\"
DATAPATH = PATH + "data\\"
FILE = f"data\\raw_data_old_{param_2}.csv"

NameError: name 'param_2' is not defined

In [5]:
def csv_reader(filename):
    df_from_csv = pd.read_csv(filename, parse_dates=['Ngày Chứng Từ'], dayfirst=True,
                              dtype={'Hóa Đơn':'str'})
    return df_from_csv
df = csv_reader(DATAPATH+'raw_data_old_20210930.csv')
df.columns = cleancols(df)

C:\Users\DELL\AppData\Local\Temp/ipykernel_13484/4228445815.py:5: DtypeWarning: Columns (5,58) have mixed types.Specify dtype option on import or set low_memory=False.
  df = csv_reader(DATAPATH+'raw_data_old_20210930.csv')


In [6]:
df_mdh_dvvc = df[['MaCongTyCN','SoDonDatHang', 'DonViGiaoHang']].copy()
df_mdh_dvvc['MaCongTyCN_SoDonDatHang'] = df['MaCongTyCN']+df['SoDonDatHang']
df_mdh_dvvc_2 = df_mdh_dvvc[['MaCongTyCN_SoDonDatHang', 'DonViGiaoHang']]
df_mdh_dvvc_2 = dropdup(df_mdh_dvvc_2,1)
mdh_dvvc_dict= df_mdh_dvvc_2.set_index('MaCongTyCN_SoDonDatHang').to_dict()['DonViGiaoHang']
x = df['DonViGiaoHang']
df['DonViGiaoHang'] = np.where(x.isna(), (df['MaCongTyCN']+df['SoDonDatHang']).map(mdh_dvvc_dict),x)

In [13]:
t1 = time.perf_counter()

print("Done merging")
# Transforming df
print('Transforming df', datetime.now())
df.replace(np.nan, '', inplace=True)
print(df.shape[0])

df_dict = pd.read_csv('ten_tat_mkv.csv', index_col='TenKhuVuc')
khuvuc_dict = df_dict.to_dict()['KhuVuc']

df['TenKhuVucVietTat'] = df['TenKhuVuc'].map(khuvuc_dict)

# Google Sheet connection
print('Google Sheet connection & join data', datetime.now())
scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive',
          'https://www.googleapis.com/auth/drive.file']
jsonfile = 'D:\\data_sale\\datateam1599968716114-6f9f144b4262.json'
credentials = service_account.Credentials.from_service_account_file(jsonfile, scopes=scopes)
service = discovery.build('drive', 'v3', credentials=credentials)
# df_product
url_product='https://drive.google.com/file/d/15ClbdtRJhqF6bIDNgRLqI1wAytIoPWWR/view?usp=sharing'
file_id_product=url_product.split('/')[-2]
dwn_url_product='https://drive.google.com/uc?id=' + file_id_product
df_product = pd.read_csv(dwn_url_product)
df = df.merge(df_product[[' Mã Sản Phẩm',' Ngành Hàng']], how = 'left',left_on='MaSanPham', right_on=' Mã Sản Phẩm',suffixes=('_left', '_right'), validate="m:1")


# df spcl_mapping -> Phan Loai
url_spcl = 'https://drive.google.com/file/d/190OCbFpxOT2KCjwhjC5nSbdcmA7ScbdW/view?usp=sharing'
file_id_spcl=url_spcl.split('/')[-2]
dwn_url_spcl = 'https://drive.google.com/uc?id=' + file_id_spcl
df_spcl = pd.read_csv(dwn_url_spcl)
df = df.merge(df_spcl, how='left', left_on='MaSanPham', right_on='MaSanPham',suffixes=('_left', '_right'), validate="m:1")
df.rename(columns={'PhanLoai': 'Nhom SPCL'}, inplace=True)

# MA HCO 2
df['MaKenhPhu_MaHCO'] = df['MaKenhPhu']+df['MaPhanLoaiHCO']
url_phanloaihco = 'https://drive.google.com/file/d/1lLdheM4T5Rp5Ht4SMjxAtm4v61wwx9Xr/view?usp=sharing'
file_id_phanloaihco=url_phanloaihco.split('/')[-2]
dwn_url_phanloaihco = 'https://drive.google.com/uc?id=' + file_id_phanloaihco
df_phanloaihco = pd.read_csv(dwn_url_phanloaihco)
df = df.merge(df_phanloaihco[['ConCat','Ma HCO 2']], how='left', left_on='MaKenhPhu_MaHCO', right_on='ConCat',suffixes=('_left', '_right'), validate="m:1")
df.rename(columns={'Ma HCO 2': 'PL HCO'}, inplace=True)

# df_clean_data
df = df[df['MaSanPham'].str[:1] != 'V']
df = df[df['MaKenhKH'] != 'NB']
df['SDDH-CN'] = df['SoDonDatHang']+'.'+df['MaCongTyCN']

print(df.shape[0])

# NhomSPCL
x = np.array(df['MaSanPham'])
spcl_inlist = ['EH111',	'EH086', 'EH104', 'EH106', 'EH103', 'EH109', 'EH115', 'EH094', 'EH092', 'EH102', 'EH116',
               'EH087', 'EH108', 'EH091', 'EH112', 'EH101', 'EH117', 'VITOL 0.10%', 'EH125', 'EH124', 'OH082']
spm_inlist = [' EH115 ', 'OH072', 'OH076']
condlist = [np.isin(x, spcl_inlist), np.isin(x, spm_inlist)]
choicelist = ['SPCL', 'SP MOI']
default = 'KHAC'
df['NhomSP'] = np.select(condlist, choicelist, default=default)

# MaMien(B/N)
x = np.array(df['MaVungBH'])
condlist = [np.isin(x, ['BB', 'HN'])]
choicelist = ['MB']
default = 'MN'
df['MaMien(B/N)'] = np.select(condlist, choicelist, default=default)

df1 = df['MaNVGH'].copy()
df1.replace(r'^\s*$', np.nan, regex=True, inplace=True)
df['MaNVGH'] = df1

df['NVBH_Ao'] = np.where(df['MaNV'].isin(['MA001', 'MA002', 'QUYNHPTA']), 0, 1)
df['NVGH_Ao'] = np.where(df['MaNVGH'].isna(), 0, 1)

df['DoanhSoBanHangChuaVAT'] = df['DoanhSoChuaVAT'] * df['NVBH_Ao']
df['DoanhSoGiaoHangChuaVAT'] = df['DoanhSoChuaVAT'] * df['NVGH_Ao']

df['SoKH'] = np.where(checkdup(df,1, 'MaKHCu'),0,1)
df['SoDH'] = np.where(checkdup(df,1, 'SDDH-CN'),0,1)


# Merap/Phanam
phanam_inlist = ['EH072', 'OH057', 'OH032', 'OH047','OH016','OH058','OH060','OH061',
          'OH062','OH063','OH064','OH065','OH070','OH069','OH066','OH068','OH067','EH105','OH071','OH079','OH081']

x = np.array(df['MaSanPham'])
condlist = [np.isin(x, phanam_inlist)]
choicelist = ['Pha Nam']
default = 'Merap'
df['Merap/PhaNam'] = np.select(condlist, choicelist, default=default)

#VanPhong
df['ChiNhanh'] = df['CongTyCN'].str.split(' ').apply(lambda x: x[-2:]).apply(lambda x: ''+' '.join(x))

cols = df.columns
unwanted_cols = {'DiaChiKH','SoHopDong','MaKenhPhu_MaHCO','ConCat',' Mã Sản Phẩm', ' Ngành Hàng','Unnamed0','MaKHThue','TenKHThue','DiaChiKHThue','MaSoThue','MaKhuVuc','MaTinhKH','MaQuan/HUyen','MaPhanHangHCO','TenPhanHangHCO','TenSanPhamNB'}
cols = [ele for ele in cols if ele not in unwanted_cols]

df=df[cols]

#df to csv
df.to_csv("raw_data.csv", index=False)


now_final = datetime.now()
print("finally imported sales data %s" % now_final)
t2 = time.perf_counter()
print(f'Finished in {t2-t1} seconds')

Done merging
Transforming df 2021-10-28 16:06:17.707622
61014
Google Sheet connection & join data 2021-10-28 16:06:17.832142
61014
finally imported sales data 2021-10-28 16:06:25.409195
Finished in 7.702465800000027 seconds


In [2]:
sql = \
"""
SELECT dc.Descr as GeneralPR, 
ds.DiscID,
ds.DiscSeq,
ds.Descr,
ds.TypeDiscount,
ds.DiscIDPN,
ds.ContentInvoice,
ds.AccumulateID,
ds.ExcludePromo,
ds.DiscFor,
ds.DiscClass,
db.BreakAmt,
db.BreakQty,
db.DiscAmt,
db.Descr as DetailDescr

FROM dbo.OM_DiscSeq ds
LEFT JOIN  dbo.OM_Discount dc ON
ds.DiscID = dc.DiscID
LEFT JOIN  dbo.OM_DiscBreak db ON
ds.DiscID = db.DiscID
and ds.DiscSeq = db.DiscSeq
"""

In [3]:
df = get_ms_df(sql)

In [5]:
bq_values_insert(df, "d_schemeinfo",1)

1it [00:03,  3.08s/it]


In [ ]:
# f_temp_sales_with_promo
# d_accumulated
# d_accumulatedregis
# d_schemeinfo
# các bảng cần schedule